In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
%load_ext gams.magic
m = gams.exchange_container

In [3]:
gams.reset()
m = gams.exchange_container
df = pd.read_csv('food_nutrients.csv', index_col="Item")
df 

,Description,Brand,Calories,Serving Size,Nutrients
Item,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0 kcal,56.0 g,"Fatty acids, total trans: 0.0 g, Carbohydrate,..."
yogurt,YOGURT,Agri-Service LLC,94.0 kcal,170.0 ml,"Iron, Fe: 0.0 mg, Calcium, Ca: 118.0 mg, Fiber..."
cereal,CEREAL,"General Mills, Inc.",387.0 kcal,31.0 g,"Fatty acids, total polyunsaturated: 1.61 g, Ir..."
Corn,CORN,"Kcb Industries, L.L.C.",545.0 kcal,33.0 g,"Vitamin D (D2 + D3), International Units: 0.0 ..."
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0 kcal,148.0 g,"Iron, Fe: 0.73 mg, Cholesterol: 0.0 mg, Calciu..."
...,...,...,...,...,...
Ice cream,ICE CREAM,"Clover Stornetta Farms, Inc.",205.0 kcal,88.0 g,"Energy: 205.0 kcal, Carbohydrate, by differenc..."
cookies,COOKIES,KNO,441.0 kcal,100.0 g,"Fiber, total dietary: 0.0 g, Sodium, Na: 0.0 m..."
brownies,BROWNIES,TWO-BITE,460.0 kcal,50.0 g,"Iron, Fe: 2.88 mg, Fiber, total dietary: 0.0 g..."


In [4]:
df['Calories'] = df['Calories'].str.extract('(\d+\.\d+)').astype(float)

In [5]:
df['Serving Size'] = df['Serving Size'].str.extract('(\d+\.\d+)').astype(float)

In [6]:
def extract_nutrients(row):
    nutrients = {}
    items = re.split(', ', row['Nutrients'])
    for item in items:
        match = re.match(r'(.+?): (.+?) (.+)', item)
        if match:
            nutrient, value = match.groups()[:2]
            nutrients[nutrient] = float(value) if '.' in value else int(value)
    return pd.Series(nutrients)

# Apply the function to each row and concatenate the result with the original DataFrame
df = pd.concat([df, df.apply(extract_nutrients, axis=1)], axis=1)

# Drop the original 'Nutrients' column
df = df.drop('Nutrients', axis=1)

In [7]:
df.head()

,Description,Brand,Calories,Serving Size,Alanine,Arginine,Ash,Aspartic acid,Ca,Caffeine,...,other,soluble,total,total ascorbic acid,total dietary,total including NLEA,total monounsaturated,total polyunsaturated,total saturated,total trans
Item,,,,,,,,,,,,,,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0,56.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,NaN,0.89,0.0
yogurt,YOGURT,Agri-Service LLC,94.0,170.0,NaN,NaN,NaN,NaN,118.0,NaN,...,NaN,NaN,NaN,0.7,0.0,15.88,NaN,NaN,1.76,NaN
cereal,CEREAL,"General Mills, Inc.",387.0,31.0,NaN,NaN,NaN,NaN,323.0,NaN,...,48.0,NaN,NaN,19.4,6.5,29.03,1.61,1.61,0.00,0.0
Corn,CORN,"Kcb Industries, L.L.C.",545.0,33.0,NaN,NaN,NaN,NaN,61.0,NaN,...,NaN,NaN,NaN,NaN,6.1,24.24,NaN,NaN,4.55,0.0
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0,148.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,2.4,2.0,0.68,NaN,NaN,0.00,0.0


In [8]:
df.rename(columns={'by difference': 'Carbs'}, inplace=True)
df.rename(columns={'Total lipid (fat)': 'Fat'}, inplace=True)
df.rename(columns={'Ca': 'Calcium'}, inplace=True)
df.rename(columns={'Vitamin D (D2 + D3)': 'Vitamin D'}, inplace=True)
df.rename(columns={'Serving Size': 'Serving'}, inplace=True)

In [9]:
df = df[["Calories", "Serving", 'Protein', "Carbs", "Fat", "Calcium"]]


In [10]:
np.random.seed(42)  # Setting a seed for reproducibility
cost_values = np.random.normal(5, 1, len(df))

df['Price'] = cost_values

In [11]:
df_limited = df.head(10)

In [12]:
df.fillna(0, inplace=True)

In [13]:
df.head()

,Calories,Serving,Protein,Carbs,Fat,Calcium,Price
Item,,,,,,,
chicken,107.0,56.0,21.43,0.00,1.79,0.0,5.496714
yogurt,94.0,170.0,2.94,15.88,2.94,118.0,4.861736
cereal,387.0,31.0,6.45,83.87,3.23,323.0,5.647689
Corn,545.0,33.0,6.06,54.55,33.33,61.0,6.523030
Potatoes,74.0,148.0,2.70,16.89,0.00,0.0,4.765847


In [14]:
# Item = m.addSet('Item',records=df_limited.index)
# p = m.addParameter('price',[Item],records=df_limited[["Price"]].to_numpy())
# cal = m.addParameter('calories',[Item],records=df_limited[["Calories"]].to_numpy())
# pro = m.addParameter('protein',[Item],records=df_limited[["Protein"]].to_numpy())
# carbs = m.addParameter('carbs',[Item],records=df_limited[["Carbs"]].to_numpy())
# fat = m.addParameter('fat',[Item],records=df_limited[["Fat"]].to_numpy())
# calcium = m.addParameter('calcium',[Item],records=df_limited[["Calcium"]].to_numpy())
# display(calcium.records)

,Item,value
0,yogurt,118.0
1,cereal,323.0
2,Corn,61.0
3,Green beans,47.0
4,Carrots,26.0
5,Tomatoes,18.0
6,Onions,17.0


In [29]:
Item = m.addSet('Item',records=df.index)
p = m.addParameter('price',[Item],records=df[["Price"]].to_numpy())
cal = m.addParameter('calories',[Item],records=df[["Calories"]].to_numpy())
pro = m.addParameter('protein',[Item],records=df[["Protein"]].to_numpy())
carbs = m.addParameter('carbs',[Item],records=df[["Carbs"]].to_numpy())
fat = m.addParameter('fat',[Item],records=df[["Fat"]].to_numpy())
calcium = m.addParameter('calcium',[Item],records=df[["Calcium"]].to_numpy())
display(calcium.records)

,Item,value
0,yogurt,118.0
1,cereal,323.0
2,Corn,61.0
3,Green beans,47.0
4,Carrots,26.0
...,...,...
65,pita bread,154.0
66,frozen vegetables,150.0
67,Ice cream,68.0
68,Pizza rolls,132.0


In [30]:
cost = m.addVariable('cost','free',description="total cost")
x = m.addVariable('x','positive',["Item"],description="number of servings for each item")


In [56]:
%%gams
scalar
    requirements /3000/;

alias(Item, i);
    
equations
    obj "minimize total meal cost"
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint;
    
obj..
    cost =e= sum(Item, price(Item)*x(Item));
    
cost_constraint..
    cost =l= 100;
    
cal_constraint_up..
    sum(Item, x(Item) * calories(Item)) =l= 3500;
    
cal_constraint_low..
    sum(Item, x(Item) * calories(Item)) =g= 3000;
    
protein_constraint_up..
    sum(i, x(i) * protein(i)) =l= 73;
    
protein_constraint_low..
    sum(i, x(i) * protein(i)) =g= 45;
    
carb_constraint_up..
    sum(i, x(i) * carbs(i)) =l= 503;
    
carb_constraint_low..
    sum(i, x(i) * carbs(i)) =g= 308;
    
fat_constraint_up..
    sum(i, x(i) * fat(i)) =l= 120;
    
fat_constraint_low..
    sum(i, x(i) * fat(i)) =g= 69;
    
calcium_constraint_up..
    sum(i, x(i) * calcium(i)) =l= 200;
    
calcium_constraint_low..
    sum(i, x(i) * calcium(i)) =g= 100;
    
# x.lo(i) = 0;
 
model plan /all/;
solve plan using mip minimizing cost;



,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),18.0772,12,94,MIP,CPLEX,0


In [54]:
x.records[x.records["level"] != 0]

,uni,level,marginal,lower,upper,scale
19,Bananas,1.535935,0.0,0.0,inf,1.0
33,Pretzels,0.211356,0.0,0.0,inf,1.0
35,Trail mix,2.322532,0.0,0.0,inf,1.0
36,Beef jerky,0.181610,0.0,0.0,inf,1.0
60,Rice,0.444677,0.0,0.0,inf,1.0
